In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
#import seaborn as sns
plt.rcParams['figure.figsize'] = 15, 12

In [3]:
d = pd.read_csv('data/DREAM5_PBM_Data_TrainingSet.txt', sep='\t')

In [4]:
d.head()

,TF_Id,ArrayType,Sequence,Signal_Mean,Background_Mean,Signal_Median,Background_Median,Signal_Std,Background_Std,Flag
0,Cebpb,HK,AAAAAACAACAGGAGGGCATCATGGAGCTGTCCAGCCTGTGTGAAA...,1508.7972,264.8014,1557.5,183.0,393.2445,249.7754,0
1,Cebpb,HK,AAAAAACAGCCGGATCACAATTTTGCCGAGAGCGACCTGTGTGAAA...,13630.7324,371.0260,14858.0,245.0,6451.2436,445.4670,0
2,Cebpb,HK,AAAAAACGTCCGGTACACCCCGTTCGGCGGCCCAGCCTGTGTGAAA...,1523.0000,265.7928,1571.5,194.5,439.6682,211.7982,0
3,Cebpb,HK,AAAAAACTCTAGACCTTTAGCCCATCGTTGGCCAACCTGTGTGAAA...,4992.8461,430.4496,5150.0,233.0,1629.2899,507.5433,0
4,Cebpb,HK,AAAAAAGAACAACCGGATAACACCCTTACAGCACACCTGTGTGAAA...,3539.8117,449.4453,3823.5,226.5,1063.9028,551.7675,0


The task is to predict the signal mean given the type of protein, array type and the sequence. I am pretty sure that one hot encoding the sequence would completely do an injustice to the richer dynamics of genes. 

An RNN, maybe? The sequence length is 60, would it be able to capture that long range dependencies?

In [5]:
d.Flag.value_counts()

0    1584778
1      32342
Name: Flag, dtype: int64

In [6]:
d = d[d.Flag == 0]

In [7]:
d.shape

(1584778, 10)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X = d[['TF_Id','ArrayType','Sequence']]
y = d[['Signal_Mean']]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [11]:
X_train.shape

(1109344, 3)

In [12]:
y_train.shape

(1109344, 1)

How naive can you go? 

I am just ignoring all the sequence information, but I will count the statistics inside DNA sequence.

In [13]:
d['A_freq'] = d.Sequence.str.count('A')
d['C_freq'] = d.Sequence.str.count('C')
d['T_freq'] = d.Sequence.str.count('T')
d['G_freq'] = d.Sequence.str.count('G')

In [14]:
d.head()

,TF_Id,ArrayType,Sequence,Signal_Mean,Background_Mean,Signal_Median,Background_Median,Signal_Std,Background_Std,Flag,A_freq,C_freq,T_freq,G_freq
0,Cebpb,HK,AAAAAACAACAGGAGGGCATCATGGAGCTGTCCAGCCTGTGTGAAA...,1508.7972,264.8014,1557.5,183.0,393.2445,249.7754,0,18,13,14,15
1,Cebpb,HK,AAAAAACAGCCGGATCACAATTTTGCCGAGAGCGACCTGTGTGAAA...,13630.7324,371.0260,14858.0,245.0,6451.2436,445.4670,0,18,14,15,13
2,Cebpb,HK,AAAAAACGTCCGGTACACCCCGTTCGGCGGCCCAGCCTGTGTGAAA...,1523.0000,265.7928,1571.5,194.5,439.6682,211.7982,0,13,19,14,14
3,Cebpb,HK,AAAAAACTCTAGACCTTTAGCCCATCGTTGGCCAACCTGTGTGAAA...,4992.8461,430.4496,5150.0,233.0,1629.2899,507.5433,0,16,16,18,10
4,Cebpb,HK,AAAAAAGAACAACCGGATAACACCCTTACAGCACACCTGTGTGAAA...,3539.8117,449.4453,3823.5,226.5,1063.9028,551.7675,0,22,16,13,9


In [15]:
# normalize
d['A_freq'] = d.A_freq / 60
d['C_freq'] = d.C_freq / 60
d['T_freq'] = d.T_freq / 60
d['G_freq'] = d.G_freq / 60

In [16]:
d['TF_Id'] = d['TF_Id'].astype('category')
d['ArrayType'] = d['ArrayType'].astype('category')

In [17]:
d= pd.get_dummies(data=d, columns=['TF_Id', 'ArrayType'])

In [18]:
d.head()

,Sequence,Signal_Mean,Background_Mean,Signal_Median,Background_Median,Signal_Std,Background_Std,Flag,A_freq,C_freq,...,TF_Id_Mecp2,TF_Id_Nr2c1,TF_Id_Pou3f1,TF_Id_Sox14,TF_Id_Sp1,TF_Id_Tbx3,TF_Id_Tcf3,TF_Id_Zscan20,ArrayType_HK,ArrayType_ME
0,AAAAAACAACAGGAGGGCATCATGGAGCTGTCCAGCCTGTGTGAAA...,1508.7972,264.8014,1557.5,183.0,393.2445,249.7754,0,0.300000,0.216667,...,0,0,0,0,0,0,0,0,1,0
1,AAAAAACAGCCGGATCACAATTTTGCCGAGAGCGACCTGTGTGAAA...,13630.7324,371.0260,14858.0,245.0,6451.2436,445.4670,0,0.300000,0.233333,...,0,0,0,0,0,0,0,0,1,0
2,AAAAAACGTCCGGTACACCCCGTTCGGCGGCCCAGCCTGTGTGAAA...,1523.0000,265.7928,1571.5,194.5,439.6682,211.7982,0,0.216667,0.316667,...,0,0,0,0,0,0,0,0,1,0
3,AAAAAACTCTAGACCTTTAGCCCATCGTTGGCCAACCTGTGTGAAA...,4992.8461,430.4496,5150.0,233.0,1629.2899,507.5433,0,0.266667,0.266667,...,0,0,0,0,0,0,0,0,1,0
4,AAAAAAGAACAACCGGATAACACCCTTACAGCACACCTGTGTGAAA...,3539.8117,449.4453,3823.5,226.5,1063.9028,551.7675,0,0.366667,0.266667,...,0,0,0,0,0,0,0,0,1,0


In [19]:
X = d.drop(['Signal_Mean','Sequence','Background_Mean','Signal_Median','Background_Median','Signal_Std','Background_Std','Flag'], axis=1)
y = d[['Signal_Mean']]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [21]:
X_train.columns

Index([u'A_freq', u'C_freq', u'T_freq', u'G_freq', u'TF_Id_Cebpb',
       u'TF_Id_Egr2', u'TF_Id_Esr1', u'TF_Id_Foxj2', u'TF_Id_Foxo1',
       u'TF_Id_Foxo3', u'TF_Id_Foxo4', u'TF_Id_Foxp1', u'TF_Id_Foxp2',
       u'TF_Id_Gmeb2', u'TF_Id_Irf2', u'TF_Id_Junb', u'TF_Id_Mecp2',
       u'TF_Id_Nr2c1', u'TF_Id_Pou3f1', u'TF_Id_Sox14', u'TF_Id_Sp1',
       u'TF_Id_Tbx3', u'TF_Id_Tcf3', u'TF_Id_Zscan20', u'ArrayType_HK',
       u'ArrayType_ME'],
      dtype='object')

In [22]:
X_train.shape

(1109344, 26)

In [23]:
20 + 2 + 4

26

## linear regression?

In [24]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [25]:
lr = LinearRegression()

In [26]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [27]:
from sklearn.metrics import r2_score

In [28]:
r2_score?

In [29]:
r2_score(y_test, lr.predict(X_test))

0.29088274264024949

In [30]:
param = {'alpha':[0.1,0.06,0.07,0.08,0.09,0.01,0.02,0.03,0.04,0.05,0.5,1,10,100,1000]}

In [31]:
from sklearn.model_selection import GridSearchCV

In [74]:
lrclf = GridSearchCV(Ridge(),param_grid=param,n_jobs=-1,cv=10)

In [75]:
lrclf.fit(X_train,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'alpha': [0.1, 0.06, 0.07, 0.08, 0.09, 0.01, 0.02, 0.03, 0.04, 0.05, 0.5, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [76]:
lrclf.best_score_

0.29116334681808764

In [77]:
lrclf.best_params_

{'alpha': 0.1}

In [ ]:
lrclf = GridSearchCV(Lasso(),param_grid=param,n_jobs=-1,cv=10)

In [ ]:
lrclf.fit(X_train,y_train)

In [ ]:
lrclf.best_score_

In [ ]:
lrclf.best_params_

Well, that wasn't bad for a naive model. The competetion had around 0.6 on final validation set.